# Dynamic Memory Allocation

At any given time a program has access to a fixed amount of memory. When it needs more memory it has to ask the OS. This is known as Dynamic Memory Allocation. It is a three step process:

1. Request memory from the OS via a system call like `alloc()` for UNIX.
2. Use the allocated memory in the program.
3. Release the memeory that isn't needed back to the OS via `free()` for UNIX.

### The Allocator

There is an intermediary between the program and the OS called the Allocator which is a program that is embedded in your program behind the scenes. It often performs optimizations that avoid lots of work within the OS and CPU. The follwing is the output from running the program `particle_generator` in the repo. It allocated memory of different sizes on the heap and keeps track of the time taken(in ns). Notice that the larger memory allocations do tend to take longer than shorter ones, it’s not guaranteed. 

In [4]:
let bytes_req = [5, 48, 9, 9, 19, 15, 16, 1, 12, 70, 22, 44, 94, 188, 96, 10, 68, 22, 44, 92, 184, 95, 190, 190, 126, 252, 504, 15]; 
let time_taken_ns = [620, 460, 500, 430, 200, 160, 80, 150, 430, 180, 350, 160, 340, 210, 100, 90, 140, 60, 90, 200, 3970, 130, 230, 200, 270, 320, 170, 150];

In [5]:
// TODO('Draw scatter plot using the above data')

### Some General Strategies To Minimize Heap Allocations

1. Using arrays of uninitialized objects. Keep in mind when using arrays with objects it can be a very dangerous strategy because you’re circumventing Rust’s lifetime checks.
2. Using an allocator that is tuned for your application’s access memory profile.
3. Investigate `arena::Arena` and `arena::TypedArena`. These allow objects to be created on the fly, but `alloc()` and `free()` are only called when the arena is created and destroyed.

# Virtual Memory

### Terminology Related To Virtual Memory

1. Page: A fixed-size block of words of real memory. Typically 4 KB in size for 64-bit operating systems.
2. Word: Any type that is size of a pointer. This corresponds to the width of the CPU’s registers. In Rust, `usize` and `isize` are word-length types.
3. Page fault: An error raised by the CPU when a valid memory address is requested that is not currently in physical RAM.This signals to the OS that at least one page must be swapped back into memory.
4. Swapping: Migrating a page of memory stored temporarily on disk from main memory upon request.
5. Virtual memory: The program’s view of its memory. All data accessible to a program is provided in its address space by the OS.
6. Real memory: The operating system’s view of the physical memory available on the system. 
7. Page table: The data structure maintained by the OS to manage translating from virtual to real memory.
8. Segment: A block within virtual memory. Virtual memory is divided into blocks to minimize the space required to translate between virtual and physical addresses.
9. Segmentation fault: An error raised by the CPU when an illegal memory address is requested.
10. MMU: A component of the CPU that manages memory address translation. Maintains a cache of recently translated addresses (called the TLB), which stands for the translation lookaside buffer, although that terminology has fallen from fashion.

### Forming An Intuition For Memory

Intuitively, a program’s memory is a series of bytes that starts at location 0 and ends at location n. If a program reports 100 KB of RAM usage, it would seem that n would be somewhere near 100,000. The following program tests that:

In [6]:
let mut n_nonzero = 0;

// creates 10000 variables on the stack
for i in 0..10000 {
    let ptr = i as *const u8; // create a raw pointer to inspect raw mem addresses
    let byte_at_addr = unsafe {
        *ptr // deref the pointer to read the value
    };
    if byte_at_addr != 0 {
        n_nonzero += 1; 
    }
}
println!("non-zero bytes in memory: {}", n_nonzero);

Segmentation fault.
   0: backtrace::capture::Backtrace::new
   1: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
   2: _OSAtomicTestAndClearBarrier
   3: _run_user_code_2
   4: _run_user_code_2
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
   8: std::sys_common::backtrace::__rust_begin_short_backtrace
   9: std::rt::lang_start::{{closure}}
  10: core::ops::function::impls::<impl core::ops::function::FnOnce<A> for &F>::call_once
             at /rustc/d5a82bbd26e1ad8b7401f6a718a9c57c96905483/library/core/src/ops/function.rs:606:13
      std::panicking::try::do_call
             at /rustc/d5a82bbd26e1ad8b7401f6a718a9c57c96905483/library/std/src/panicking.rs:483:40
      std::panicking::try
             at /rustc/d5a82bbd26e1ad8b7401f6a718a9c57c96905483/library/std/src/panicking.rs:447:19
      std::panic::catch_unwind
             at /rustc/d5a82bbd26e1ad8b7401f6a718a9c57c96905483/library/std/src/panic.rs:137:14
 

Error: Subprocess terminated with status: signal: 6 (SIGABRT)

The above program crashes because it is attempting to dereference a NULL pointer. When i equals 0, ptr can’t really be dereferenced. How about we start from 1 to avoid the NULL pointer issue?

In [7]:
let mut n_nonzero = 0;

for i in 1..10000 {
    let ptr = i as *const u8; 
    let byte_at_addr = unsafe {
        *ptr 
    };
    if byte_at_addr != 0 {
        n_nonzero += 1; 
    }
}
println!("non-zero bytes in memory: {}", n_nonzero);

Segmentation fault.
   0: backtrace::capture::Backtrace::new
   1: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
   2: _OSAtomicTestAndClearBarrier
   3: _run_user_code_2
   4: _run_user_code_2
   6: evcxr::runtime::runtime_hook
   5: evcxr::runtime::Runtime::run_loop
   7: evcxr_jupyter::main
   8: std::sys_common::backtrace::__rust_begin_short_backtrace
   9: std::rt::lang_start::{{closure}}
  10: core::ops::function::impls::<impl core::ops::function::FnOnce<A> for &F>::call_once
             at /rustc/d5a82bbd26e1ad8b7401f6a718a9c57c96905483/library/core/src/ops/function.rs:606:13
      std::panicking::try::do_call
             at /rustc/d5a82bbd26e1ad8b7401f6a718a9c57c96905483/library/std/src/panicking.rs:483:40
      std::panicking::try
             at /rustc/d5a82bbd26e1ad8b7401f6a718a9c57c96905483/library/std/src/panicking.rs:447:19
             at /rustc/d5a82bbd26e1ad8b7401f6a718a9c57c96905483/library/std/src/panic.rs:137:14
      std::panic::catch_unwind
 

Error: Subprocess terminated with status: signal: 6 (SIGABRT)

Upon running the above program we're getting what's called a `Segmentation fault`.

## Segmentation Fault

Segmentation faults are generated when the CPU and OS detect that your program is attempting to access memory regions that they aren’t entitled to. Memory regions are divided into segments hence the name.

To fix this issue rather than attempting to scan through bytes, let’s look for the addresses of things that we know exist.

In [12]:
static GLOBAL: i32 = 1000;

fn noop() -> *const i32 {
    let noop_local = 12345;
    &noop_local as *const i32
}

let local_str = "a";
let local_int = 123;
let boxed_str = Box::new('b');
let boxed_int = Box::new(789);
let fn_int = noop();

println!("GLOBAL:    {:p}", &GLOBAL as *const i32);
println!("local_str: {:p}", local_str as *const str);
println!("local_int: {:p}", &local_int as *const i32);
println!("boxed_int: {:p}", Box::into_raw(boxed_int));
println!("boxed_str: {:p}", Box::into_raw(boxed_str));
println!("fn_int: {:p}", fn_int);

GLOBAL:    0x1058bf304
local_str: 0x1058bf37c
local_int: 0x16b9d9fac
boxed_int: 0x6000031b0080
boxed_str: 0x6000031b0090
fn_int: 0x16b9da01c


Notice that these addresses seem to be scattered across a wide range. In one instance of runnnig the program, `local_str` had the address `0x1058bf37c` which translates to  1,761,786,620 and `local_int` had `0x16b9d9fac` 3,793,758,956. The largest address in that run was 1,152,921,505,896. So despite our program requiring only a few KBs of memeory, some varibles live in giant locations. These are virtual addresses.

**SIDE NOTE**: The OS reserves half the address space for itself.

1. Some memory addresses are illegal.
2. Memory addresses are not arbitrary. Although values seem to be spread quite far apart within the address space, values are clustered together within pockets.

### Translating Virtual Addresses To Physical Address

Accessing data in a program requires virtual addresses—the only addresses that the program itself has access to. These get translated into physical addresses. This process involves the program, the OS, the CPU, the RAM hardware, and occasionally hard drives and other devices. The CPU is responsible for performing this translation, but the OS stores the instructions.

CPUs contain a memory management unit (MMU) that is designed for this one job. For every running program, every virtual address is mapped to a physical address. Those instructions are stored at a predefined address in memory as well. That means, in the worst case, every attempt at accessing memory addresses incurs two memory lookups. But it’s possible to avoid the worst case. The CPU maintains a cache of recently translated addresses. It has its own (fast) memory to speed up accessing memory.

**Programmers optimizing for performance need to keep data structures lean and avoid deeply nested structures.**

Virtual addresses are grouped into blocks called pages, which are typically 4 KB in size. This practice avoids the need to store a translation mapping for every single variable in every program. Having a uniform size for each page also assists in avoiding a phenomenon known as memory fragmentation, where pockets of empty, yet unusable, space appear within available RAM.

### Benefits Of Virtual Memory

1. Having a virtual address space allows the OS to overallocate. Programs that ask for more memory than the machine can physically provide are able to be accommodated.
2. Inactive memory pages can be swapped to disk in a byte-for-byte manner until it’s requested by the active program. Swapping is often used during periods of high contention for memory but can also be used more generally.
3. Other size optimizations such as compression can be performed. A program sees its memory intact. Behind the scenes, the OS compresses the program’s wasteful data usage.
4. Programs are able to share data quickly. If your program requests a large block of zeroes, say, for a newly created array, the OS might point you towards a page filled with zeroes that is currently being used by three other programs. None of the programs are aware that the others are looking at the same physical mem- ory, and the zeroes have different positions within their virtual address space.
5. Paging can speed up the loading of shared libraries. If a shared library is already loaded by another program, the OS can avoid loading it into memory twice by pointing the new program to the old data.
6. Paging adds security between programs. If an attempt is made to write to a read-only page, the OS termi- nates the program.

### How To Effectively Use Virtual Memory

Making effective use of the virtual memory system in day-to-day programs requires thinking about how data is represented in RAM. Here are some guidelines:

1. Keep hot working portions of your program within 4 KB of size. This maintains fast lookups.
2. If 4 KB is unreasonable for your application, then the next target to keep under is 4 KB * 100. That rough guide should mean that the CPU can maintain its translation cache (the TLB) in good order to support your program.
3. Avoid deeply nested data structures with pointer spaghetti. If a pointer points to another page, then performance suffers.
4. Test the ordering of your nested loops. CPUs read small blocks of bytes, known as a cache line, from the RAM hardware. When processing an array, you can take advantage of this by investigating whether you are doing column-wise or row- wise operations.

### Virtualization Makes Virtual Memory Management Worse

If you’re running an app inside a virtual machine, the hypervisor must also translate addresses for its guest operating systems. This is why many CPUs ship with virtualization support, which can reduce this extra overhead. Running containers within virtual machines adds another layer of indirection and, therefore, latency. For bare-metal performance, run apps on bare metal.